<a href="https://colab.research.google.com/github/mille055/AIPI540_individual_project/blob/main/notebooks/AIPI540_IP_fusion_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [57]:

#!git clone 'https://github.com/mille055/AIPI540_individual_project.git'

In [58]:
# !pip install pydicom==2.1.2
# !pip install monai seaborn

In [154]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
import torchvision
import pydicom
import monai
import pickle
import glob
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import seaborn as sns

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import time
import os
import copy

import monai
from monai.data import DataLoader, ImageDataset
from monai.transforms import AddChannel, Compose, RandRotate90, Resize, ScaleIntensity, EnsureType
from pydicom.dataset import Dataset as DcmDataset
from pydicom.tag import BaseTag as DcmTag
from pydicom.multival import MultiValue as DcmMultiValue

import warnings
warnings.filterwarnings("ignore")

In [168]:
#local imports
import sys
sys.path.append('/Users/cmm/Documents/GitHub/AIPI540_individual_project/scripts/')

### local imports ###
from config import new_dict
from config import abd_label_dict, val_list, train_val_split_percent, random_seed, data_transforms
from config import sentence_encoder, series_description_column, text_label
from utils import *

# from AIPI540_individual_project.scripts.train_pixel_model import train_model
# from AIPI540_individual_project.scripts.train_text_model import load_text_data, train_text_model, list_incorrect_text_predictions
# from AIPI540_individual_project.scripts.utils import *

ImportError: cannot import name 'new_dict' from 'config' (/Users/cmm/Documents/GitHub/AIPI540_individual_project/scripts/config.py)

{'train': Compose(
    ToPILImage()
    Resize(size=299, interpolation=PIL.Image.BILINEAR)
    CenterCrop(size=(299, 299))
    Grayscale(num_output_channels=3)
    ToTensor()
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
), 'val': Compose(
    ToPILImage()
    Resize(size=299, interpolation=PIL.Image.BILINEAR)
    CenterCrop(size=(299, 299))
    Grayscale(num_output_channels=3)
    ToTensor()
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
), 'test': Compose(
    ToPILImage()
    Resize(size=299, interpolation=PIL.Image.BILINEAR)
    CenterCrop(size=(299, 299))
    Grayscale(num_output_channels=3)
    ToTensor()
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
)}


# Load Datasets


In [22]:
# for colab
# train_datafile = '/content/AIPI540_individual_project/data/X_train02282023.pkl'
# test_datafile = '/content/AIPI540_individual_project/data/X_test02282023.pkl'

In [61]:
# for local
train_datafile = './data/trainfiles.csv'
test_datafile = './data/testfiles.csv'

In [64]:
def load_csv_dataset(train_file, test_file, val = True, val_lists = None):
    train_df = pd.read_csv(train_file)
    test_df = pd.read_csv(test_file)
    train_df.drop('Unnamed: 0', axis=1, inplace=True)
    test_df.drop('Unnamed: 0', axis=1, inplace=True)
    if val:
        if val_lists:
            val_df = train_df[train_df.patientID.isin(val_lists)]
            train_df = train_df[~train_df.index.isin(val_df.index)] 
        else:
            train_set, val_set = next(GroupShuffleSplit(test_size=.20, n_splits=1, random_state = 42).split(train_df, groups=train_df['patientID']))
            train_df, val_df = train_set, val_set
        return train_df, val_df, test_df

    else: 
        return train_df, test_df


In [76]:
train, val, test = load_csv_dataset(train_datafile, test_datafile, val_lists = val_list)
train = shorten_df(train)
val = shorten_df(val)
test = shorten_df(test)

In [77]:
train_s

,label,patientID,series,file_info
0,8,104,18(35-68),/volumes/cm7/Abdominal_MRI_dataset_split/train...
1,8,104,18(1-34),/volumes/cm7/Abdominal_MRI_dataset_split/train...
2,16,104,20,/volumes/cm7/Abdominal_MRI_dataset_split/train...
3,9,104,9,/volumes/cm7/Abdominal_MRI_dataset_split/train...
4,9,104,11,/volumes/cm7/Abdominal_MRI_dataset_split/train...
...,...,...,...,...
1407,7,94,3,/volumes/cm7/Abdominal_MRI_dataset_split/train...
1408,2,94,12,/volumes/cm7/Abdominal_MRI_dataset_split/train...
1409,2,94,13,/volumes/cm7/Abdominal_MRI_dataset_split/train...
1410,19,94,5,/volumes/cm7/Abdominal_MRI_dataset_split/train...


In [108]:
sample_dcm = pydicom.dcmread(train.iloc[1410].file_info)
sample_dcm

Dataset.file_meta -------------------------------
(0002, 0000) File Meta Information Group Length  UL: 212
(0002, 0001) File Meta Information Version       OB: b'\x00\x01'
(0002, 0002) Media Storage SOP Class UID         UI: MR Image Storage
(0002, 0003) Media Storage SOP Instance UID      UI: 8.7.0.6.0.3.68126.2.1.3.76116323207412881310707585688286
(0002, 0010) Transfer Syntax UID                 UI: Implicit VR Little Endian
(0002, 0012) Implementation Class UID            UI: 1.2.826.0.1.3680043.2.1143.107.104.103.115.2.6.3
(0002, 0013) Implementation Version Name         SH: 'ANONYMIZED'
-------------------------------------------------
(0008, 0005) Specific Character Set              CS: 'ISO_IR 100'
(0008, 0008) Image Type                          CS: ['ORIGINAL', 'PRIMARY', 'M', 'NORM', 'DIS2D']
(0008, 0016) SOP Class UID                       UI: MR Image Storage
(0008, 0018) SOP Instance UID                    UI: 1.4.3.6.2.2.80771.2.1.3.76116323207412881310707585688286
(0008,

In [113]:
!pip show monai

Name: monai
Version: 1.1.0
Summary: AI Toolkit for Healthcare Imaging
Home-page: https://monai.io/
Author: MONAI Consortium
Author-email: monai.contact@gmail.com
License: Apache License 2.0
Location: /Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages
Requires: torch, numpy
Required-by: 


In [112]:
# Read in a DICOM file
reader = monai.data.DICOMImageReader()
image, metadata = reader.read(sample_dcm)

# Convert the image to a NumPy array
image_array = np.array(image)

print(metadata)

AttributeError: module 'monai.data' has no attribute 'DICOMImageReader'

In [118]:
# Read in a DICOM file
ds = pydicom.dcmread(train.iloc[1410].file_info)

# List all of the attributes on the FileDataset object
print(dir(ds))

['AcquisitionMatrix', 'AcquisitionNumber', 'AngioFlag', 'BitsAllocated', 'BitsStored', 'BodyPartExamined', 'Columns', 'EchoNumbers', 'EchoTime', 'EchoTrainLength', 'FlipAngle', 'HighBit', 'ImageOrientationPatient', 'ImagePositionPatient', 'ImageType', 'ImagedNucleus', 'ImagingFrequency', 'InPlanePhaseEncodingDirection', 'InstanceNumber', 'LargestImagePixelValue', 'MRAcquisitionType', 'MagneticFieldStrength', 'Modality', 'NumberOfAverages', 'NumberOfPhaseEncodingSteps', 'PatientAge', 'PatientName', 'PatientPosition', 'PatientSex', 'PatientSize', 'PatientWeight', 'PercentPhaseFieldOfView', 'PercentSampling', 'PhotometricInterpretation', 'PixelBandwidth', 'PixelData', 'PixelRepresentation', 'PixelSpacing', 'PositionReferenceIndicator', 'RepetitionTime', 'Rows', 'SAR', 'SOPClassUID', 'SOPInstanceUID', 'SamplesPerPixel', 'ScanOptions', 'ScanningSequence', 'SequenceVariant', 'SeriesDescription', 'SeriesInstanceUID', 'SeriesNumber', 'SliceLocation', 'SliceThickness', 'SmallestImagePixelValue'

In [125]:
len(dicom_cols), len(dir(ds)), len(mylist)

(32, 141, 22)

In [126]:
mylist = []
badlist = []
for col in dicom_cols:
    if col in dir(ds):
        mylist.append(col)
    else:
        badlist.append(col)
mylist, badlist

(['StudyInstanceUID',
  'MagneticFieldStrength',
  'SeriesInstanceUID',
  'SeriesNumber',
  'SeriesDescription',
  'BodyPartExamined',
  'AcquisitionNumber',
  'InstanceNumber',
  'SOPClassUID',
  'ImageOrientationPatient',
  'EchoTime',
  'EchoTrainLength',
  'RepetitionTime',
  'SequenceVariant',
  'ScanOptions',
  'ScanningSequence',
  'MRAcquisitionType',
  'ImageType',
  'PixelSpacing',
  'SliceThickness',
  'PhotometricInterpretation',
  'AngioFlag'],
 ['PatientID',
  'StudyID',
  'StudyDescription',
  'Manufacturer',
  'ManufacturerModelName',
  'SequenceName',
  'InversionTime',
  'TriggerTime',
  'ContrastBolusAgent',
  'DiffusionBValue'])

In [103]:
dicom_cols = [
    'PatientID',
    # Study info
    'StudyInstanceUID',
    'StudyID',
    'StudyDescription', # to filter on "MRI BRAIN WITH AND WITHOUT CONTRAST" in some cases
    'Manufacturer',
    'ManufacturerModelName',
    'MagneticFieldStrength',
    # Series info
    'SeriesInstanceUID',
    'SeriesNumber',
    'SeriesDescription', # needed for labeling series
    'SequenceName', # may be used for labeling series
    'BodyPartExamined', # to filter on "HEAD" or "BRAIN"
    'AcquisitionNumber',
    # Image info and features
    'InstanceNumber', # i.e. image number
    'SOPClassUID', # to filter on "MR Image Storage"
    'ImageOrientationPatient', # to calculate slice orientation (e.g. axial, coronal, sagittal)
    'EchoTime',
    'InversionTime',
    'EchoTrainLength',
    'RepetitionTime',
    'TriggerTime',
    'SequenceVariant',
    'ScanOptions',
    'ScanningSequence',
    'MRAcquisitionType',
    'ImageType',
    'PixelSpacing',
    'SliceThickness',
    'PhotometricInterpretation',
    'ContrastBolusAgent',
    'AngioFlag', # addition to list from paper
    'DiffusionBValue' # addition to list from paper
]

In [104]:
dicom_cols

['PatientID',
 'StudyInstanceUID',
 'StudyID',
 'StudyDescription',
 'Manufacturer',
 'ManufacturerModelName',
 'MagneticFieldStrength',
 'SeriesInstanceUID',
 'SeriesNumber',
 'SeriesDescription',
 'SequenceName',
 'BodyPartExamined',
 'AcquisitionNumber',
 'InstanceNumber',
 'SOPClassUID',
 'ImageOrientationPatient',
 'EchoTime',
 'InversionTime',
 'EchoTrainLength',
 'RepetitionTime',
 'TriggerTime',
 'SequenceVariant',
 'ScanOptions',
 'ScanningSequence',
 'MRAcquisitionType',
 'ImageType',
 'PixelSpacing',
 'SliceThickness',
 'PhotometricInterpretation',
 'ContrastBolusAgent',
 'AngioFlag',
 'DiffusionBValue']

In [89]:
fns = train.file_info.tolist()

# Create an empty list to store the DICOM metadata
data = []

# Loop over the DICOM file names and extract the metadata
for file_name in file_names:
    # Read in the DICOM file
    ds = pydicom.dcmread(file_name)
    
    d = {}
    for col in dicom_cols:
        d.key = col
        d.value = ds['col']
    # Store the metadata in a dictionary
    d = {
        'patient_id': patient_id,
        'study_date': study_date,
        # ... add other metadata as needed ...
    }
    
    # Append the dictionary to the data list
    data.append(d)

# Create a DataFrame from the data list
df = pd.DataFrame(data)
This code reads in a list of DICOM file names, then loops over the files and extracts the metadata of interest using pydicom. It stores the metadata in a dictionary, then appends the dictionary to a list of dictionaries (data). Finally, it creates a Pandas DataFrame from the list of dictionaries using pd.DataFrame(data). You can modify this code to extract the specific metadata that you are interested in from your DICOM files.








AttributeError: type object 'DataFrame' has no attribute 'from_dicoms'

In [43]:
train_label_df = train[['fname_x', 'filename', 'Parent_folder', 'patientID', 'exam', 'series', 'SeriesDescription_y', 'plane', 'seq_label', 'contrast', 'original_label_code', 'ap_label_code']]
train_label_df.rename(columns={'SeriesDescription_y':'SeriesDescription'}, inplace=True)
train_label_df

,fname_x,filename,Parent_folder,patientID,exam,series,SeriesDescription,plane,seq_label,contrast,original_label_code,ap_label_code
0,Abdominal MRI data/104/exam1/18 (35-68)/0038.dcm,0038.dcm,Abdominal MRI data,104,exam1,18(35-68),ax diff_TRACEW,ax,dwi,0,8,8
1,Abdominal MRI data/104/exam1/18 (1-34)/0010.dcm,0010.dcm,Abdominal MRI data,104,exam1,18(1-34),ax diff_TRACEW,ax,dwi,0,8,8
2,Abdominal MRI data/104/exam1/20/0010.dcm,0010.dcm,Abdominal MRI data,104,exam1,20,ax equilibrium,ax,equilibrium,1,16,16
3,Abdominal MRI data/104/exam1/9/0010.dcm,0010.dcm,Abdominal MRI data,104,exam1,9,ax haste bh,ax,t2,0,9,9
4,Abdominal MRI data/104/exam1/11/0010.dcm,0010.dcm,Abdominal MRI data,104,exam1,11,ax t2 triggerred,ax,t2,0,9,9
...,...,...,...,...,...,...,...,...,...,...,...,...
1729,Abdominal MRI data/094/exam1/3/0010.dcm,0010.dcm,Abdominal MRI data,94,exam1,3,cor haste,cor,t2,0,7,7
1730,Abdominal MRI data/094/exam1/12/0010.dcm,0010.dcm,Abdominal MRI data,94,exam1,12,ax dyn,ax,early dynamic,1,2,2
1731,Abdominal MRI data/094/exam1/13/0010.dcm,0010.dcm,Abdominal MRI data,94,exam1,13,ax dyn,ax,early dynamic,1,2,2
1732,Abdominal MRI data/094/exam1/5/0010.dcm,0010.dcm,Abdominal MRI data,94,exam1,5,ax dixon_opp,ax,opposed phase,0,19,19


In [45]:
test_label_df = test[['fname_x', 'filename', 'Parent_folder', 'patientID', 'exam', 'series', 'SeriesDescription_y', 'plane', 'seq_label', 'contrast', 'original_label_code', 'ap_label_code']]
test_label_df.rename(columns={'SeriesDescription_y':'SeriesDescription'}, inplace=True)
test_label_df

,fname_x,filename,Parent_folder,patientID,exam,series,SeriesDescription,plane,seq_label,contrast,original_label_code,ap_label_code
0,Abdominal MRI data/102/exam1/16 (1-40)/0010.dcm,0010.dcm,Abdominal MRI data,102,exam1,16(1-40),ax diff_TRACEW,ax,dwi,0,8,8
1,Abdominal MRI data/102/exam1/16 (41-80)/0073.dcm,0073.dcm,Abdominal MRI data,102,exam1,16(41-80),ax diff_TRACEW,ax,dwi,0,8,8
2,Abdominal MRI data/102/exam1/18/0010.dcm,0010.dcm,Abdominal MRI data,102,exam1,18,ax equilibrium new,ax,equilibrium,1,16,16
3,Abdominal MRI data/102/exam1/11/0010.dcm,0010.dcm,Abdominal MRI data,102,exam1,11,ax dyn new,ax,early dynamic,1,3,2
4,Abdominal MRI data/102/exam1/7/0010.dcm,0010.dcm,Abdominal MRI data,102,exam1,7,ax dixon_W,ax,unknown,0,25,25
...,...,...,...,...,...,...,...,...,...,...,...,...
427,Abdominal MRI data/058/exam1/16 (1-36)/0010.dcm,0010.dcm,Abdominal MRI data,58,exam1,16(1-36),ax diff_TRACEW,ax,dwi,0,8,8
428,Abdominal MRI data/058/exam1/2/0010.dcm,0010.dcm,Abdominal MRI data,58,exam1,2,cor haste,cor,t2,0,7,7
429,Abdominal MRI data/058/exam1/13/0010.dcm,0010.dcm,Abdominal MRI data,58,exam1,13,ax dyn new,ax,early dynamic,1,2,2
430,Abdominal MRI data/058/exam1/5/0010.dcm,0010.dcm,Abdominal MRI data,58,exam1,5,ax dixon_in,ax,in phase,0,14,14


In [49]:
# for colab
# trainids = pd.read_csv('/content/AIPI540_individual_project/data/trainfiles.csv')
# for local
trainids = pd.read_csv('./data/trainfiles.csv')


In [50]:
trainids.iloc[6].file_info

'/volumes/cm7/Abdominal_MRI_dataset_split/train/104/exam1/18 (35-68)/0059.dcm'

In [51]:
val_df = pd.read_pickle('/content/AIPI540_individual_project/data/val_dicom_data.pkl')

FileNotFoundError: [Errno 2] No such file or directory: '/content/AIPI540_individual_project/data/val_dicom_data.pkl'

In [38]:
val_df

,label_code_x,predictions,patientID,series,img_label_code,GT label,prediction label,probability,original_label_code,SeriesDescription_y,plane,seq_label,contrast,label_code_y,final_prediction
0,8,8,41,16(37-72),8,dwi,dwi,0.976494,8,ax diff_TRACEW,ax,dwi,0,8,8
1,30,30,41,18,16,T1 post con,T1 post con fat sat,0.956994,16,ax equilibrium new,ax,equilibrium,1,16,16
2,30,30,41,11,3,T1 post con,T1 post con fat sat,0.956994,3,ax dyn new,ax,early dynamic,1,3,3
3,25,25,41,7,25,T1 fat sat,T1 fat sat,0.957891,25,ax dixon_W,ax,unknown,0,25,25
4,11,11,41,6,11,dixon fat,dixon fat,0.955352,11,ax dixon_F,ax,unknown,0,11,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
349,9,9,66,12,9,t2,t2,0.977097,9,ax t2 resp trig,ax,t2,0,9,9
350,7,7,66,2,17,t2,t2,0.914605,7,cor haste,cor,t2,0,7,7
351,11,11,66,5,11,dixon fat,dixon fat,0.834404,11,ax dixon top_F_FIL,ax,unknown,0,11,11
352,0,0,66,14,0,adc,adc,0.449541,0,ax diff_ADC,ax,dwi,0,0,0


In [ ]:
### does not have the filename colums.... will look for another
